# 個人課題_1
### （課題内容）

* Github の Google が管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
* リポジトリ名、主要な言語、スターの数

* 上記のデータを保存するための DB を作成し、スクレイピングしたデータを保存する
* ※ Github の API は使わないこと
* ※ ちゃんと time.sleep(1) 入れてね

* 保存したデータを SELECT 文で表示する

* Google Classroom でプログラムを提出すること



#  ステップ_1 (google ウエブスクレピング)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3
import time

#  GitHubからデータを取得
print("ステップ1: GitHubからデータを取得")

# ブラウザを開く
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Googleのリポジトリページを開く
url = "https://github.com/google?tab=repositories"
print(f"ページを開いています: {url}\n")
driver.get(url)
print("ページの読み込みを待っています...")
time.sleep(10)

# リポジトリのリンクを全部取得
repo_links = driver.find_elements(By.CSS_SELECTOR, "a[itemprop='name codeRepository']")
print(f"リポジトリが {len(repo_links)} 個見つかりました\n")

# データを保存するリスト
repositories = []

# 各リポジトリの情報を取得
for i, link in enumerate(repo_links, 1):
    
    # リポジトリ名を取得
    repo_name = link.text.strip()
    
    # 親要素を探す（リポジトリの情報が入っている部分）
    parent = link
    for _ in range(5):
        parent = parent.find_element(By.XPATH, "..")
        if parent.tag_name == "li":
            break
    
    # 言語を取得
    try:
        lang_elem = parent.find_element(By.CSS_SELECTOR, "span[itemprop='programmingLanguage']")
        language = lang_elem.text.strip()
    except:
        language = "不明"
    
    # スター数を取得
    try:
        star_elem = parent.find_element(By.CSS_SELECTOR, "a[href*='/stargazers']")
        star_text = star_elem.text.strip().replace(",", "")
        
        # "8k" みたいな表記を数字に変換
        if "k" in star_text.lower():
            stars = int(float(star_text.lower().replace("k", "")) * 1000)
        else:
            stars = int(star_text)
    except:
        stars = 0
    
    # リストに追加
    repositories.append((repo_name, language, stars))
    
    # 画面に表示
    print(f"[{i}] {repo_name}")
    print(f"    言語: {language}")
    print(f"    スター: {stars}")
    print()
    time.sleep(1)

# ブラウザを閉じる
driver.quit()

print(f"取得完了。 {len(repositories)} 個のリポジトリを取得しました\n")


ステップ1: GitHubからデータを取得
ページを開いています: https://github.com/google?tab=repositories

ページの読み込みを待っています...
リポジトリが 10 個見つかりました

[1] device-infra
    言語: Java
    スター: 58

[2] dotprompt
    言語: Python
    スター: 323

[3] skia
    言語: C++
    スター: 10250

[4] angle
    言語: C++
    スター: 3840

[5] orbax
    言語: Python
    スター: 452

[6] langfun
    言語: Python
    スター: 873

[7] filonov
    言語: Python
    スター: 10

[8] tunix
    言語: Python
    スター: 1888

[9] perfetto
    言語: C++
    スター: 4963

[10] osv-scanner
    言語: Go
    スター: 8063

取得完了。 10 個のリポジトリを取得しました



In [ ]:
#  データベースに接続
path = ''
db_name = 'github_repositories.db'

# DB 接続オブジェクトの作成
conn = sqlite3.connect(path + db_name)

# SQL を実行するためのカーソルオブジェクトを取得
cur = conn.cursor()

# テーブルを作成（古いテーブルがあれば削除）
sql = "DROP TABLE IF EXISTS repositories;"
cur.execute(sql)

sql = """
CREATE TABLE repositories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    repo_name TEXT,
    language TEXT,
    stars INTEGER
);
"""
cur.execute(sql)
for repo in repositories:
    repo_name, language, stars = repo

    sql = f"""
    INSERT INTO repositories (repo_name, language, stars)
    VALUES ('{repo_name}', '{language}', {stars});
    """
    cur.execute(sql)
conn.commit()



In [64]:
print("ステップ3: データベースに保存")

for repo_name, language, stars in repositories:
    cur.execute(
        "INSERT INTO repositories (repo_name, language, stars) VALUES (?, ?, ?)",
        (repo_name, language, stars)
    )

conn.commit()
print(f"{len(repositories)} 件のデータを保存しました\n")

ステップ3: データベースに保存
10 件のデータを保存しました



In [65]:
# SELECT文でデータを表示
print("ステップ4: SELECT文でデータを表示")

cur.execute("""
    SELECT id, repo_name, language, stars
    FROM repositories
    ORDER BY stars DESC
""")
rows = cur.fetchall()

print(f"\n{'ID':<5} {'リポジトリ名':<40} {'言語':<15} {'スター数':>10}")

for row in rows:
    id_num, name, language, stars = row
    print(f"{id_num:<5} {name:<40} {language:<15} {stars:>10}")

print(f"合計: {len(rows)} 件\n")
conn.close()


ステップ4: SELECT文でデータを表示

ID    リポジトリ名                                   言語                    スター数
3     skia                                     C++                  10250
10    osv-scanner                              Go                    8063
9     perfetto                                 C++                   4963
4     angle                                    C++                   3840
8     tunix                                    Python                1888
6     langfun                                  Python                 873
5     orbax                                    Python                 452
2     dotprompt                                Python                 323
1     device-infra                             Java                    58
7     filonov                                  Python                  10
合計: 10 件

